In [2]:
import os
import sys
sys.path.append("BEE_WEATHER_DATA")
from BEE_WEATHER_DATA import BROODMINDER_GET, AMBIENT_GET, BROODMINDER_PANDAS
import pandas as pd
from glob import glob
import re

hive_name = "New Left Hive"

def read_hive(hive_name):
    directory = "Broodminder/"
    Hive = pd.DataFrame()
    res = [f for f in glob(directory + "*.csv") if hive_name in f]
    for f in res:
        Hive = pd.read_csv(f)
    return Hive

print(read_hive(hive_name))
    # NewLeftHive.drop('App', axis=1, inplace=True)
    # NewLeftHive.drop('Record_Type', axis=1, inplace=True)
    # NewLeftHive.drop('DownloadTimeStamp', axis=1, inplace=True)
    # NewLeftHive.drop('UTC_TimeStamp', axis=1, inplace=True)
    # NewLeftHive.drop('Local_TimeStamp', axis=1, inplace=True)
    # NewLeftHive.drop('Metric', axis=1, inplace=True)
    # NewLeftHive.drop(' WeightLRaw', axis=1, inplace=True)
    # NewLeftHive.drop(' WeightRRaw', axis=1, inplace=True)
    # NewLeftHive = NewLeftHive[['Unix_Time','Device','Hive_Position','Sample','Battery','Temperature','Humidity','Scaled_Weight',' Weight_Scale_Factor',' Weight',' WeightL',' WeightR']]
    # send2trash(f)

# filename = "Left Hive_weather_20[a-zA-Z0-9\-\_\.]*.csv"
# for f in glob('/Users/kevinhardin/Downloads/'+filename):
#     NewHiveWeather = pd.read_csv(f)
#     NewHiveWeather.drop('DownloadTimeStamp', axis=1, inplace=True)
#     NewHiveWeather.drop('UTC_TimeStamp', axis=1, inplace=True)
#     NewHiveWeather.drop('Local_TimeStamp', axis=1, inplace=True)
#     NewHiveWeather.drop('Metric', axis=1, inplace=True)
#     send2trash(f)

# filename = "Right Hive_combined_readings_20[a-zA-Z0-9\-\_\.]*.csv"
# for f in glob('/Users/kevinhardin/Downloads/'+filename):
#     NewRightHive = pd.read_csv(f)
#     NewRightHive.drop('App', axis=1, inplace=True)
#     NewRightHive.drop('Record_Type', axis=1, inplace=True)
#     NewRightHive.drop('DownloadTimeStamp', axis=1, inplace=True)
#     NewRightHive.drop('UTC_TimeStamp', axis=1, inplace=True)
#     NewRightHive.drop('Local_TimeStamp', axis=1, inplace=True)
#     NewRightHive.drop('Metric', axis=1, inplace=True)
#     NewRightHive.drop(' WeightLRaw', axis=1, inplace=True)
#     NewRightHive.drop(' WeightRRaw', axis=1, inplace=True)
#     NewRightHive = NewRightHive[['Unix_Time','Device','Hive_Position','Sample','Battery','Temperature','Humidity','Scaled_Weight',' Weight_Scale_Factor',' Weight',' WeightL',' WeightR']]
#     send2trash(f)

# for f in glob(filename1):
#     OldLeftHive = pd.read_csv(f)
#     fields = OldLeftHive.columns
#     new_entries = pd.concat([OldLeftHive,NewLeftHive]).astype(str).drop_duplicates(subset=['Unix_Time'], keep='last').reset_index(drop=True)
#     new_entries.to_csv(filename1, mode='w', index=False, header=True)

# for f in glob(filename2):
#     OldHiveWeather = pd.read_csv(f)
#     fields = OldHiveWeather.columns
#     new_entries = pd.concat([OldHiveWeather,NewHiveWeather]).astype(str).drop_duplicates(subset=['Unix_Time'], keep='last').reset_index(drop=True)
#     new_entries.to_csv(filename2, mode='w', index=False, header=True)

# for f in glob(filename3):
#     OldRightHive = pd.read_csv(f)
#     fields = OldRightHive.columns
#     new_entries = pd.concat([OldRightHive,NewRightHive]).astype(str).drop_duplicates(subset=['Unix_Time'], keep='last').reset_index(drop=True)
#     new_entries.to_csv(filename3, mode='w', index=False, header=True)

# LeftHive = pd.read_csv(filename1)
# HiveWeather = pd.read_csv(filename2)
# RightHive = pd.read_csv(filename3)

# # LeftHive resampled and interpolated to five minutes
# LeftHive['Unix_Time'] = LeftHive['Unix_Time'].astype('int64')
# LeftHive['Unix_Time'] = pd.to_datetime(LeftHive['Unix_Time'], unit='s', origin='unix')

# # RightHive resampled and interpolated to five minutes
# RightHive['Unix_Time'] = RightHive['Unix_Time'].astype('int64')
# RightHive['Unix_Time'] = pd.to_datetime(RightHive['Unix_Time'], unit='s', origin='unix')

# # HiveWeather resampled and interpolated to five minutes
# HiveWeather['Unix_Time'] = HiveWeather['Unix_Time'].astype('int64')
# HiveWeather['Unix_Time'] = pd.to_datetime(HiveWeather['Unix_Time'], unit='s', origin='unix')

# RightUpper = RightHive[RightHive['Device'] == '56:0B:D1']
# RightLower = RightHive[RightHive['Device'] == '47:13:0C']
# RightScale = RightHive[RightHive['Device'] == '49:02:8D']
# LeftUpper = LeftHive[LeftHive['Device'] == '56:0B:D0']
# LeftLower = LeftHive[LeftHive['Device'] == '47:13:E4']
# LeftScale = LeftHive[LeftHive['Device'] == '49:02:8C']

# # Should downsampling be "mean" instead of asfreq?  Need to specify and align start date/time? 1652133991

# #     # Sort based on device
# #     LeftUpper = LeftHive[LeftHive['Device'] == '56:0B:D0']
# #     LeftLower = LeftHive[LeftHive['Device'] == '47:13:E4']
# #     LeftScale = LeftHive[LeftHive['Device'] == '49:02:8C']

# # LeftUpper resampled and interpolated to five minutes (no weight measurements)
# LeftUpper = LeftUpper.set_index('Unix_Time',drop=False,inplace=False)
# LeftUpper.drop('Device', axis=1, inplace=True)
# LeftUpper.drop('Hive_Position', axis=1, inplace=True)
# LeftUpper.drop('Sample', axis=1, inplace=True)
# LeftUpper.drop('Scaled_Weight', axis=1, inplace=True)
# LeftUpper.drop(' Weight_Scale_Factor', axis=1, inplace=True)
# LeftUpper.drop(' Weight', axis=1, inplace=True)
# LeftUpper.drop(' WeightL', axis=1, inplace=True)
# LeftUpper.drop(' WeightR', axis=1, inplace=True)
# LeftUpper_upsampled = LeftUpper.resample('1s').mean()
# LeftUpper_interpolated = LeftUpper_upsampled[LeftUpper_upsampled.columns[1::]].interpolate(method='linear')
# LeftUpper_downsampled = LeftUpper_interpolated.resample('300s').mean()
# # LeftUpper_downsampled.iloc[:, 0]

# # LeftLower resampled and interpolated to five minutes (no weight or humidity measurements)
# LeftLower = LeftLower.set_index('Unix_Time',drop = False)
# LeftLower.drop('Device', axis=1, inplace=True)
# LeftLower.drop('Hive_Position', axis=1, inplace=True)
# LeftLower.drop('Sample', axis=1, inplace=True)
# LeftLower.drop('Scaled_Weight', axis=1, inplace=True)
# LeftLower.drop(' Weight_Scale_Factor', axis=1, inplace=True)
# LeftLower.drop(' Weight', axis=1, inplace=True)
# LeftLower.drop(' WeightL', axis=1, inplace=True)
# LeftLower.drop(' WeightR', axis=1, inplace=True)
# LeftLower.drop('Humidity', axis=1, inplace=True)
# LeftLower_upsampled = LeftLower.resample('1s').mean()
# LeftLower_interpolated = LeftLower_upsampled[LeftLower_upsampled.columns[1::]].interpolate(method='linear')
# LeftLower_downsampled = LeftLower_interpolated.resample('300s').mean()

#  # LeftScale resampled and interpolated to five minutes (no humidity measurements)
# LeftScale = LeftScale.set_index('Unix_Time',drop = False)
# LeftScale.drop('Device', axis=1, inplace=True)
# LeftScale.drop('Hive_Position', axis=1, inplace=True)
# LeftScale.drop('Sample', axis=1, inplace=True)
# LeftScale.drop('Humidity', axis=1, inplace=True)
# LeftScale_upsampled = LeftScale.resample('1s').mean()
# LeftScale_interpolated = LeftScale_upsampled[LeftScale_upsampled.columns[1::]].interpolate(method='linear')
# LeftScale_downsampled = LeftScale_interpolated.resample('300s').mean()
# LeftAvg = LeftScale_downsampled['Scaled_Weight'].resample('1d').mean()

# #     RightUpper = RightHive[RightHive['Device'] == '56:0B:D1']
# #     RightLower = RightHive[RightHive['Device'] == '47:13:0C']
# #     RightScale = RightHive[RightHive['Device'] == '49:02:8D']

# # RightUpper resampled and interpolated to five minutes (no weight measurements)
# RightUpper = RightUpper.set_index('Unix_Time',drop = False)
# RightUpper.drop('Device', axis=1, inplace=True)
# RightUpper.drop('Hive_Position', axis=1, inplace=True)
# RightUpper.drop('Sample', axis=1, inplace=True)
# RightUpper.drop('Scaled_Weight', axis=1, inplace=True)
# RightUpper.drop(' Weight_Scale_Factor', axis=1, inplace=True)
# RightUpper.drop(' Weight', axis=1, inplace=True)
# RightUpper.drop(' WeightL', axis=1, inplace=True)
# RightUpper.drop(' WeightR', axis=1, inplace=True)
# RightUpper_upsampled = RightUpper.resample('1s').mean()
# RightUpper_interpolated = RightUpper_upsampled[RightUpper_upsampled.columns[1::]].interpolate(method='linear')
# RightUpper_downsampled = RightUpper_interpolated.resample('300s').mean()

# # RightLower resampled and interpolated to five minutes (no weight or humidity measurements)
# RightLower = RightLower.set_index('Unix_Time',drop = False)
# RightLower.drop('Device', axis=1, inplace=True)
# RightLower.drop('Hive_Position', axis=1, inplace=True)
# RightLower.drop('Sample', axis=1, inplace=True)
# RightLower.drop('Scaled_Weight', axis=1, inplace=True)
# RightLower.drop(' Weight_Scale_Factor', axis=1, inplace=True)
# RightLower.drop(' Weight', axis=1, inplace=True)
# RightLower.drop(' WeightL', axis=1, inplace=True)
# RightLower.drop(' WeightR', axis=1, inplace=True)
# RightLower.drop('Humidity', axis=1, inplace=True)
# RightLower_upsampled = RightLower.resample('1s').mean()
# RightLower_interpolated = RightLower_upsampled[RightLower_upsampled.columns[1::]].interpolate(method='linear')
# RightLower_downsampled = RightLower_interpolated.resample('300s').mean()

# # RightScale resampled and interpolated to five minutes (no humidity measurements)
# RightScale = RightScale.set_index('Unix_Time',drop = False)
# RightScale.drop('Device', axis=1, inplace=True)
# RightScale.drop('Hive_Position', axis=1, inplace=True)
# RightScale.drop('Sample', axis=1, inplace=True)
# RightScale.drop('Humidity', axis=1, inplace=True)
# RightScale_upsampled = RightScale.resample('1s').mean()
# RightScale_interpolated = RightScale_upsampled[RightScale_upsampled.columns[1::]].interpolate(method='linear')
# RightScale_downsampled = RightScale_interpolated.resample('300s').mean()
# RightAvg = RightScale_downsampled['Scaled_Weight'].resample('1d').mean()

# HiveWeather = HiveWeather.set_index('Unix_Time',drop = False)
# HiveWeather_upsampled = HiveWeather.resample('1s').mean()
# HiveWeather_interpolated = HiveWeather_upsampled[HiveWeather_upsampled.columns[1::]].interpolate(method='linear')
# HiveWeather_downsampled = HiveWeather_interpolated.resample('300s').mean()

      App  Rssi    Device     Hive_Position  Record_Type        UTC_TimeStamp  \
0    H|60    88  49:02:8D  Scale Under Hive  Logged_Data   02/25/2024 7:33 AM   
1    H|60    28  56:0B:D0       Upper Brood  Logged_Data   02/25/2024 7:53 AM   
2    H|60    34  47:20:22       Lower Brood  Logged_Data   02/25/2024 7:53 AM   
3    H|60    99  49:02:8D  Scale Under Hive  Logged_Data   02/25/2024 8:33 AM   
4    H|60    42  47:20:22       Lower Brood  Logged_Data   02/25/2024 8:53 AM   
..    ...   ...       ...               ...          ...                  ...   
564  H|60    44  47:20:22       Lower Brood  Logged_Data  03/03/2024 10:52 PM   
565  H|60    96  49:02:8D  Scale Under Hive  Logged_Data  03/03/2024 11:22 PM   
566  H|60    40  47:20:22       Lower Brood  Logged_Data  03/03/2024 11:52 PM   
567  H|60    99  49:02:8D  Scale Under Hive  Logged_Data  03/04/2024 12:22 AM   
568  H|60    24  56:0B:D0       Upper Brood  Logged_Data  03/04/2024 12:22 AM   

         Local_TimeStamp   

In [28]:
print(directory[0:-1] + "\/" + "New Left Hive_combined_readings_20[a-zA-Z0-9\-\_\.]*.csv")

Broodminder\/\/New Left Hive_combined_readings_20[a-zA-Z0-9\-\_\.]*.csv
